In [12]:
def read_split(path:str,size:float):
  data = pd.read_csv(path)
  X = data.drop(columns=['y'])
  y = data.y
  return train_test_split(X,y, test_size=size, random_state=123)

In [13]:
import pandas as pd
import os
import copy
import glob
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
import pickle as pick


import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from  xgboost import XGBClassifier as xgb
from sklearn.model_selection import GridSearchCV

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import TomekLinks 
from sklearn.metrics import balanced_accuracy_score 
import sklearn.metrics as met

import scipy.stats as stats

Falta - LR

In [14]:
thetas = np.linspace(0,1,101)

In [14]:
paths = glob.glob('./OneDrive_2022-10-05/datasets DEFINITIVO/*.csv')
paths

['./OneDrive_2022-10-05/datasets DEFINITIVO\\Bank Marketing (1).csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\card_transdata.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\ecoli-0-1-4-6_vs_5.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\ecoli-0-1-4-7_vs_5-6.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\ecoli-0-2-3-4_vs_5.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\ecoli-0-3-4_vs_5.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\ecoli-0-6-7_vs_3-5.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\fertility_Diagnosis.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\glass-0-1-4-6_vs_2.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\glass4.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\Internet Advertisements (1).csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\led7digit-0-2-4-5-6-7-8-9_vs_1.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\page-blocks0.csv',
 './OneDrive_2022-10-05/datasets DEFINITIVO\\qsar_oral_toxicity (1).csv',
 './OneDrive_202

In [26]:
!mkdir data

Ya existe el subdirectorio o el archivo data.


In [15]:
for idx, path in enumerate(paths):
   os.rename(path,f'./data/data{idx}.csv')

In [15]:
paths = glob.glob('./data/*.csv')
paths

['./data\\data0.csv',
 './data\\data1.csv',
 './data\\data10.csv',
 './data\\data11.csv',
 './data\\data12.csv',
 './data\\data13.csv',
 './data\\data14.csv',
 './data\\data15.csv',
 './data\\data16.csv',
 './data\\data17.csv',
 './data\\data18.csv',
 './data\\data19.csv',
 './data\\data2.csv',
 './data\\data20.csv',
 './data\\data21.csv',
 './data\\data3.csv',
 './data\\data4.csv',
 './data\\data5.csv',
 './data\\data6.csv',
 './data\\data7.csv']

In [16]:
models = []
models.append(('Tree',DecisionTreeClassifier()))
models.append(('RF',RandomForestClassifier()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('XGB',xgb()))

In [17]:
parameters = []
parameters.append(('Tree',{'min_samples_leaf':[20,30,40,50],
                           'min_impurity_decrease':[0,0.01,0.03,0.05],
                           'ccp_alpha':[0,0.01,0.03,0.05],
                           'random_state':[123]}))

#class_weight='balanced'

parameters.append(('RF',{'n_estimators':[500],
                         'min_samples_leaf':[20,30,40,50],
                         'min_impurity_decrease':[0,0.01,0.03,0.05],
                         'ccp_alpha':[0,0.01,0.03,0.05],
                         'oob_score':[False, True],
                         'random_state':[123]}))

parameters.append(('KNN',{'n_neighbors':[3,5,7,11,21],
                          'algorithm':['ball_tree','kd_tree'],
                          'p':[1,2],
                          'leaf_size':[20,30,40]}))

parameters.append(('XGB',{'learning_rate': [0.3, 0.5, 0.7],
                          'min_child_weight': [1,3,5],
                          'subsample': [0.5,0.6,0.7,0.8],
                          'verbosity': [0],
                          'colsample_bytree': [0.7,0.8,0.9,1],
                          'n_estimators': [100,300,500],
                          'random_state':[123]}))
parameters = dict(parameters)

In [18]:
dict_models = {'Tree':{'t':[],'nt':[], 'size':[]},
              'RF':{'t':[],'nt':[],'size':[]},
              'KNN':{'t':[],'nt':[],'size':[]},
              'XGB':{'t':[],'nt':[],'size':[]}
              }

In [19]:
dict_models1 = copy.deepcopy(dict_models);
dict_models2 = copy.deepcopy(dict_models);
dict_models3 = copy.deepcopy(dict_models);
dict_models4 = copy.deepcopy(dict_models);

In [21]:
for path in paths:
  X_train, X_test, y_train, y_test = read_split(path,0.2)
  ros = SMOTE(random_state=123)
  x_ros, y_ros = ros.fit_resample(X_train,y_train)

  results = []
  names = []
  for name, model in models:
    model_grid = GridSearchCV(model,
                 parameters[name],
                 cv = 5,
                 n_jobs = 5,
                 verbose=True,
                 scoring='balanced_accuracy')
    model_grid.fit(x_ros, y_ros)
    print(model_grid.best_params_)
    # print(model_grid.best_score_)
    model.parameters = model_grid.best_params_
    #cv_results = cross_val_score(model,x_ros, y_ros, cv=5,scoring='balanced_accuracy')
    #results.append(cv_results)
    #names.append(name)
    #msg= '%s: %f (%f)' %(name, cv_results.mean(), cv_results.std())
    #print(msg)
    model.fit(x_ros,y_ros)
    print('ros: ',balanced_accuracy_score(y_test, model.predict(X_test)))
    dict_models1[name]['t'].append(balanced_accuracy_score(y_test, model.predict(X_test)))
    dict_models1[name]['size'].append(len(X_test))
    model.fit(X_train,y_train)
    print('noRos:',balanced_accuracy_score(y_test, model.predict(X_test)))
    dict_models1[name]['nt'].append(balanced_accuracy_score(y_test, model.predict(X_test)))
    print('******************')
  # alg = str(input('Ingrese el Algoritmo deseado: '))
  # print(dict_models[alg])


Fitting 5 folds for each of 64 candidates, totalling 320 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'random_state': 123}
ros:  0.7143749788241058
noRos: 0.6987788587454725
******************
Fitting 5 folds for each of 128 candidates, totalling 640 fits


KeyboardInterrupt: 

In [47]:
path

'./data\\data10.csv'

In [66]:
dict_models1

{'Tree': {'t': [0.6631205673758864], 'nt': [0.6482269503546099], 'size': [62]},
 'RF': {'t': [], 'nt': [], 'size': []},
 'KNN': {'t': [], 'nt': [], 'size': []},
 'XGB': {'t': [], 'nt': [], 'size': []}}

In [59]:
for path in paths:
  X_train, X_test, y_train, y_test = read_split(path,0.2)
  rus = TomekLinks()
  x_rus, y_rus = rus.fit_resample(X_train,y_train)

  results = []
  names = []
  for name, model in models:
    model_grid = GridSearchCV(model,
                 parameters[name],
                 cv = 5,
                 n_jobs = 5,
                 verbose=True,
                 scoring='balanced_accuracy')
    model_grid.fit(x_rus, y_rus)
    print(model_grid.best_params_)
    # print(model_grid.best_score_)
    model.parameters = model_grid.best_params_
    cv_results = cross_val_score(model,x_rus, y_rus, cv=5,scoring='balanced_accuracy')
    results.append(cv_results)
    names.append(name)
    msg= '%s: %f (%f)' %(name, cv_results.mean(), cv_results.std())
    print(msg)
    model.fit(x_rus,y_rus)
    print(balanced_accuracy_score(y_test, model.predict(X_test)))
    dict_models2[name]['t'].append(balanced_accuracy_score(y_test, model.predict(X_test)))
    model.fit(X_train,y_train)
    print('noRus:',balanced_accuracy_score(y_test, model.predict(X_test)))
    dict_models2[name]['nt'].append(balanced_accuracy_score(y_test, model.predict(X_test)))
    print('******************')
  # alg = str(input('Ingrese el Algoritmo deseado: '))
  # print(dict_models[alg])

Fitting 5 folds for each of 64 candidates, totalling 320 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 40, 'random_state': 123}
Tree: 0.533429 (0.074021)
0.6602836879432624
noRus: 0.5709219858156028
******************
Fitting 5 folds for each of 128 candidates, totalling 640 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'n_estimators': 500, 'oob_score': False, 'random_state': 123}
RF: 0.620302 (0.062172)
0.5723404255319149
noRus: 0.5936170212765958
******************
Fitting 5 folds for each of 60 candidates, totalling 300 fits
{'algorithm': 'ball_tree', 'leaf_size': 30, 'n_neighbors': 3, 'p': 1}
KNN: 0.601515 (0.027532)
0.6815602836879433
noRus: 0.6588652482269504
******************
Fitting 5 folds for each of 432 candidates, totalling 2160 fits


C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass 

{'colsample_bytree': 1, 'learning_rate': 0.5, 'min_child_weight': 3, 'n_estimators': 100, 'random_state': 123, 'subsample': 0.5, 'verbosity': 0}


C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass 

XGB: 0.612623 (0.040460)
0.5418439716312057
noRus: 0.5617021276595745
******************
Fitting 5 folds for each of 64 candidates, totalling 320 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'random_state': 123}
Tree: 0.846669 (0.109220)
0.9939759036144578
noRus: 0.9939759036144578
******************
Fitting 5 folds for each of 128 candidates, totalling 640 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'n_estimators': 500, 'oob_score': False, 'random_state': 123}
RF: 0.860635 (0.110276)
0.9939759036144578
noRus: 0.9939759036144578
******************
Fitting 5 folds for each of 60 candidates, totalling 300 fits
{'algorithm': 'ball_tree', 'leaf_size': 40, 'n_neighbors': 5, 'p': 1}
KNN: 0.825764 (0.083082)
0.9879518072289157
noRus: 0.9879518072289157
******************
Fitting 5 folds for each of 432 candidates, totalling 2160 fits


...

C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


noRus: 0.9461478620097645
******************
Fitting 5 folds for each of 64 candidates, totalling 320 fits
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'random_state': 123}
Tree: 0.706202 (0.024440)
0.718625177954037


exception calling callback for <Future at 0x22a05e99b20 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\externals\loky\process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\multiprocessing\queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\numpy_pickle.py", line 526, in load_temporary_memmap
    obj = load(filename, mmap_mode)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\numpy_pickle.py", line 587, in load
    obj = _unpickle(fobj, filename, mmap_mode)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\numpy_pickle.py", line 506, in _unpickle
    obj = unpickler.load()
  File "C:\Users\sdariza\.co

noRus: 0.7030364043115721
******************
Fitting 5 folds for each of 128 candidates, totalling 640 fits
Unexpected exception formatting exception. Falling back to standard exception


joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\externals\loky\process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\multiprocessing\queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\numpy_pickle.py", line 526, in load_temporary_memmap
    obj = load(filename, mmap_mode)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\numpy_pickle.py", line 587, in load
    obj = _unpickle(fobj, filename, mmap_mode)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\numpy_pickle.py", line 506, in _unpickle
    obj = unpickler.load()
  File "C:\Users\sdariza\.conda\envs\mineria\lib\pickle.py", line 1212, in load
    dispatch[key[0]](self)
KeyError: 0
"""

T

In [60]:
dict_models2

{'Tree': {'t': [0.6602836879432624,
   0.9939759036144578,
   0.9206502494769033,
   0.718625177954037],
  'nt': [0.5709219858156028,
   0.9939759036144578,
   0.9172836525564676,
   0.7030364043115721]},
 'RF': {'t': [0.5723404255319149, 0.9939759036144578, 0.9505338269220451],
  'nt': [0.5936170212765958, 0.9939759036144578, 0.9461478620097645]},
 'KNN': {'t': [0.6815602836879433, 0.9879518072289157, 0.8558801437845378],
  'nt': [0.6588652482269504, 0.9879518072289157, 0.8574091957723053]},
 'XGB': {'t': [0.5418439716312057, 0.9939759036144578, 0.9631820376629647],
  'nt': [0.5617021276595745, 0.9939759036144578, 0.9461478620097645]}}

In [38]:
for path in paths:
  X_train, X_test, y_train, y_test = read_split(path,0.2)
  results = []
  names = []
  for name, model in models:
    model_grid = GridSearchCV(model,
                  parameters[name],
                  cv = 5,
                  n_jobs = 5,
                  verbose=True,
                  scoring='balanced_accuracy')
    model_grid.fit(X_train, y_train)
    probs = np.array(model_grid.predict_proba(X_train)[:,0])
    perfor=[]
    for theta in thetas:
      pred = 1*(probs>=theta)
      perfor.append(met.balanced_accuracy_score(y_train,pred))
    perfor=np.array(perfor,dtype=np.float32)
    list1 = np.array(perfor)
    k = 7
    pos = []
    for i in range(k):
        max_v = np.argmax(list1)
        x=list1[max_v]
        for j in range(1,4):
            list1[max_v-j]=-1*(list1[max_v-j]==x)+list1[max_v-j]*(list1[max_v-j]!=x)
            list1[max_v+j]=-1*(list1[max_v+j]==x)+list1[max_v+j]*(list1[max_v+j]!=x)
        pos.append(max_v)
        list1[max_v] = -1
    x=abs((thetas[pos]-0.5)*(thetas[pos]>=0.5))+thetas[pos]*(thetas[pos]<0.5)
    tOptim=pos[np.where(x==np.max(x))[0][0]]/100
    print(tOptim)
    print(model_grid.best_params_)
    model.parameters = model_grid.best_params_
    cv_results = cross_val_score(model,X_train, y_train, cv=5,scoring='balanced_accuracy')
    results.append(cv_results)
    names.append(name)
    msg= '%s: %f (%f)' %(name, cv_results.mean(), cv_results.std())
    print(msg)
    model.fit(X_train,y_train)
    probs=np.array(model.predict_proba(X_test)[:,0])
    print('noTruncated:',met.balanced_accuracy_score(y_test, model.predict(X_test)))
    print('Truncated',met.balanced_accuracy_score(y_test, 1*(probs>=tOptim)))
    dict_models3[name]['t'].append(met.balanced_accuracy_score(y_test, 1*(probs>=tOptim)))
    dict_models3[name]['nt'].append(met.balanced_accuracy_score(y_test, model.predict(X_test)))
    print('******************')

Fitting 5 folds for each of 64 candidates, totalling 320 fits
0.47
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'random_state': 123}
Tree: 0.687500 (0.192638)
noTruncated: 0.6666666666666666
Truncated 0.3333333333333333
******************
Fitting 5 folds for each of 128 candidates, totalling 640 fits
0.45
{'ccp_alpha': 0, 'min_impurity_decrease': 0, 'min_samples_leaf': 20, 'n_estimators': 500, 'oob_score': False, 'random_state': 123}
RF: 0.693845 (0.097166)
noTruncated: 0.6666666666666666
Truncated 0.3333333333333333
******************
Fitting 5 folds for each of 60 candidates, totalling 300 fits
0.94
{'algorithm': 'ball_tree', 'leaf_size': 20, 'n_neighbors': 7, 'p': 1}
KNN: 0.784470 (0.192715)
noTruncated: 0.4875
Truncated 0.5
******************
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
0.94
{'colsample_bytree': 0.8, 'learning_rate': 0.5, 'min_child_weight': 1, 'n_estimators': 100, 'random_state': 123, 'subsample': 0.8, 'verbosity': 0}
XGB

In [24]:
dict_models1

{'Tree': {'t': [0.6404255319148936], 'nt': [0.625531914893617]},
 'RF': {'t': [0.6737588652482269], 'nt': [0.5602836879432624]},
 'KNN': {'t': [0.6524822695035462], 'nt': [0.6588652482269504]},
 'XGB': {'t': [0.5964539007092198], 'nt': [0.5617021276595745]}}

In [41]:
dict_models3

{'Tree': {'t': [0.45177304964539006, 0.37446808510638296, 0.6148936170212767],
  'nt': [0.5482269503546099, 0.625531914893617, 0.6148936170212767]},
 'RF': {'t': [0.4574468085106383, 0.6524822695035462],
  'nt': [0.5829787234042554, 0.5829787234042554]},
 'KNN': {'t': [0.5, 0.5914893617021277],
  'nt': [0.6588652482269504, 0.6588652482269504]},
 'XGB': {'t': [0.5, 0.5404255319148936],
  'nt': [0.5617021276595745, 0.5617021276595745]}}

In [ ]:
X_train, X_test, y_train, y_test = read_split(paths[0],0.2)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)

In [ ]:
knn.predict(X_test)

In [ ]:
balanced_accuracy_score(y_test, knn.predict(X_test))

In [ ]:
probs = np.array(knn.predict_proba(X_train)[:,0])

In [231]:
X_train, X_test, y_train, y_test = read_split(path,0.2)
results = []
names = []
model_grid = RandomForestClassifier(random_state=8)
model_grid.fit(X_train, y_train)
probs = np.array(model_grid.predict_proba(X_train)[:,0])
perfor=[]
for theta in thetas:
  pred = 1*(probs>=theta)
  perfor.append(met.precision_score(y_train, pred,average="macro"))
perfor=np.array(perfor,dtype=np.float32)
list1 = np.array(perfor)
k = 7
pos = []
for i in range(k):
    max_v = np.argmax(list1)
    x=list1[max_v]
    for j in range(1,4):
        list1[max_v-j]=-1*(list1[max_v-j]==x)+list1[max_v-j]*(list1[max_v-j]!=x)
        list1[max_v+j]=-1*(list1[max_v+j]==x)+list1[max_v+j]*(list1[max_v+j]!=x)
    pos.append(max_v)
    list1[max_v] = -1
x=abs((thetas[pos]-0.5)*(thetas[pos]>=0.5))+thetas[pos]*(thetas[pos]<0.5)
tOptim=pos[np.where(x==np.max(x))[0][0]]/100
print(perfor)
print(tOptim)

print(met.precision_score(y_train, 1*(probs>=tOptim),average="macro"))
met.recall_score(y_train, 1*(probs>=tOptim),average="macro")
# log_loss roc_auc_score(y_train,pred, average='weighted', multi_class='ovr')

[0.3647541  0.35067874 0.34134614 0.32539684 0.31142858 0.297546
 0.28289473 0.2739726  0.26595744 0.26086956 0.2421875  0.22268908
 0.21551724 0.2079646  0.19724771 0.18867925 0.18571429 0.16666667
 0.15979381 0.15263158 0.14516129 0.125      0.10714286 0.09756097
 0.09756097 0.08227848 0.08227848 0.07692308 0.06578948 0.06
 0.05405406 0.05405406 0.04794521 0.04166667 0.03521127 0.02857143
 0.02857143 0.02857143 0.02857143 0.02857143 0.02857143 0.02857143
 0.02857143 0.02857143 0.02489973 0.02489973 0.02076949 0.02076949
 0.02076949 0.01614583 0.01614583 0.01098901 0.01098901 0.01098901
 0.01098901 0.01098901 0.01098901 0.0136612  0.0136612  0.0136612
 0.0136612  0.01630435 0.01630435 0.01891892 0.02406417 0.02406417
 0.02659575 0.03157895 0.03645833 0.04822335 0.05050505 0.05276382
 0.06372549 0.06796116 0.07004831 0.07211538 0.07619048 0.08215962
 0.08986175 0.08986175 0.09728507 0.10089686 0.10444444 0.11135371
 0.11471862 0.11637931 0.12127659 0.12288135 0.12761506 0.1307054
 0.13

C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\skl

0.24157303370786518

In [ ]:
perfor=[]
for theta in thetas:
  pred = 1*(probs>=theta)+2*(probs<theta)
  perfor.append(roc_auc_score(y_train,pred, average='weighted', multi_class='ovr'))
perfor=np.array(perfor,dtype=np.float32)
tOptim=thetas[perfor.argmax()]
print(perfor)

In [ ]:
# pd.DataFrame.from_dict(dict_models, orient='index')

In [ ]:
for key,value in dict_models1.items():
  plt.plot(dict_models1[key]['t'],'o',label=key+'t')
  plt.plot(dict_models1[key]['nt'],'o',label=key+'nt')
  plt.legend()
  plt.show()

In [121]:
parameters = []
parameters.append(('Tree',{'class_weight':['balanced'],
                           'min_samples_leaf':[20,30,40,50],
                           'min_impurity_decrease':[0,0.01,0.03,0.05],
                           'ccp_alpha':[0,0.01,0.03,0.05],
                           'random_state':[123]}))

parameters.append(('RF',{'class_weight':['balanced'],
                         'n_estimators':[500],
                         'min_samples_leaf':[20,30,40,50],
                         'min_impurity_decrease':[0,0.01,0.03,0.05],
                         'ccp_alpha':[0,0.01,0.03,0.05],
                         'oob_score':[False, True],
                         'random_state':[123]}))

parameters.append(('XGB',{'sample_type':['weighted'],
                          'learning_rate': [0.3, 0.5, 0.7],
                          'min_child_weight': [1,3,5],
                          'subsample': [0.5,0.6,0.7,0.8],
                          'verbosity' : [0],
                          'colsample_bytree': [0.7,0.8,0.9,1],
                          'n_estimators': [100,300,500],
                          'random_state':[123]}))
parameters = dict(parameters)

In [122]:
models1 = []
models1.append(('Tree',DecisionTreeClassifier()))
models1.append(('RF',RandomForestClassifier()))
models1.append(('XGB',xgb()))

In [123]:
for path in paths:
  X_train, X_test, y_train, y_test = read_split(path,0.2)
  results = []
  names = []
  for name, model in models1:
    model_grid = GridSearchCV(model,
                  parameters[name],
                  cv = 5,
                  n_jobs = 5,
                  verbose=True,
                  scoring='balanced_accuracy')
    model_grid.fit(X_train, y_train)
    model.parameters = model_grid.best_params_
    #cv_results = cross_val_score(model,X_train, y_train, cv=5,scoring='balanced_accuracy')
    #results.append(cv_results)
    #names.append(name)
    #msg= '%s: %f (%f)' %(name, cv_results.mean(), cv_results.std())
    #print(msg)
    model.fit(X_train,y_train)
    print('weighted: ',balanced_accuracy_score(y_test, model.predict(X_test)))
    dict_models4[name]['t'].append(balanced_accuracy_score(y_test, model.predict(X_test)))
    if name == 'XGB':
        model.parameters.pop('sample_type','weighted')
    else:
        model.parameters.pop('class_weight','balanced')
    model.fit(X_train,y_train)
    #pick.dump(model,open(f'./weighted_{name}.sav','wb'))
    print('no_weighted: ',balanced_accuracy_score(y_test, model.predict(X_test)))
    dict_models4[name]['nt'].append(balanced_accuracy_score(y_test, model.predict(X_test)))
    print('******************')

Fitting 5 folds for each of 64 candidates, totalling 320 fits
weighted:  0.6148936170212767
no_weighted:  0.5921985815602837
******************
Fitting 5 folds for each of 128 candidates, totalling 640 fits
weighted:  0.5829787234042554
no_weighted:  0.5723404255319149
******************
Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\sdariza\AppData\Local\Temp\4\ipykernel_10636\1257707153.py", line 12, in <cell line: 1>
    model_grid.fit(X_train, y_train)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\sklearn\model_selection\_search.py", line 891, in fit
    self._run_search(evaluate_candidates)
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\sklearn\model_selection\_search.py", line 1392, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\sklearn\model_selection\_search.py", line 838, in evaluate_candidates
    out = parallel(
  File "C:\Users\sdariza\.conda\envs\mineria\lib\site-packages\joblib\parallel.py", line 1098, in __call__
  File "C:\Users\sdariza\.conda\envs\mineria\li

In [22]:
dict_models = {'Tree':{'t':[0.525,0.8465,0.8518],'nt':[0.551,0.6565,0.851], 'size':[50,60,20]},
              'RF':{'t':[],'nt':[],'size':[]},
              'KNN':{'t':[],'nt':[],'size':[]},
              'XGB':{'t':[],'nt':[],'size':[]}
              }

In [43]:

x=[np.array(dict_models["Tree"]["t"], dtype=np.float32) * np.array(dict_models["Tree"]["size"],dtype=np.float32),np.array(dict_models["Tree"]["nt"], dtype=np.float32) * np.array(dict_models["Tree"]["size"],dtype=np.float32) ]
x=np.array(x)

In [45]:
stats.chi2_contingency(x)[1]

0.637419454871635

In [35]:
np.array(dict_models["Tree"]["t"], dtype=np.float32) * np.array(dict_models["Tree"]["size"],dtype=np.float32)

array([26.249998, 50.789997, 17.036001], dtype=float32)

In [ ]:
if min(stats.shapiro(DatosMAPETrain).pvalue,stats.shapiro(DatosMAPETest).pvalue)<0.05:
    pval=min(stats.mannwhitneyu(DatosMAPETrain,DatosMAPETest).pvalue,stats.fligner(DatosMAPETrain,DatosMAPETest).pvalue)
else:
    pval=min(stats.ttest_ind(DatosMAPETrain,DatosMAPETest).pvalue,stats.levene(DatosMAPETrain,DatosMAPETest).pvalue) 
DatosMAPE=np.concatenate((DatosMAPETrain, DatosMAPETest), axis=None)
if pval>=0.05:
  print("El modelo es consistente y tiene un MAPE de: ",DatosMAPE.mean())
pval=min(pval,sm.stats.acorr_ljungbox(DatosMAPE)[1][0])
if pval>=0.05:
  print("El modelo es válido para predecir")